In [7]:
import numpy as np
import pandas as pd

In [8]:
train = pd.read_csv('p2_train.csv')
test = pd.read_csv('p2_test.csv')

In [9]:
train.head()

,Unnamed: 0,text,label
0,7744,longlost pictur archiv celebr glamour seneg wr...,0
1,10536,advanc understand natur world respons current ...,1
2,7043,itali test western economi bear almost total s...,1
3,5930,south africa box granni juke jab way healthier...,1
4,11660,coronaviru pandem nation tragedi hundr thousan...,1


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train['text'], train['label'], test_size=0.2, random_state=42)

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout

# create a dataset with 1020 rows of T1 and T2
# T1 = ["The cat is sitting on the mat", "The dog is chasing the cat", "The bird is flying in the sky"] # list of T1 texts
# T2 = ["A feline is seated on the mat", "A canine is pursuing the feline", "A feathered creature is soaring in the heavens"] # list of T2 texts
# labels = np.array([1,1,0])

# split the data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(train['text'], train['label'], test_size=0.2, random_state=42)

# tokenization and padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
max_length = max([len(seq) for seq in X_train_seq])
max_length = max([len(seq) for seq in X_test_seq])
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

# LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=32, input_length=max_length))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(64))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# train the model
model.fit(X_train_padded, y_train, epochs=10, batch_size=32)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 5985, 32)          714880    
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 5985, 128)         20608     
                                                                 
 dropout_4 (Dropout)         (None, 5985, 128)         0         
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 64)                12352     
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 2)                

In [12]:
tokenizer.fit_on_texts(X_test)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

In [13]:
loss, acc = model.evaluate(X_test_padded, y_test)

11/11 [==============================] - 10s 876ms/step - loss: 0.7018 - accuracy: 0.4897


In [21]:
preds = model.predict(X_test_padded)
# preds = np.argmax(preds, axis=-1)
preds[0]

11/11 [==============================] - 10s 884ms/step


array([0.48056424, 0.4248256 ], dtype=float32)

In [24]:
preds_new = []
for i in preds:
  if(i[0]>i[1]):
    preds_new.append(1.0)
  else:
    preds_new.append(0.0)

print(preds_new[0])
y_test_array = np.array(y_test)
print(y_test_array[0])

1.0
1


In [25]:
from sklearn.metrics import f1_score,precision_score, recall_score, accuracy_score
f1 = f1_score(y_test, preds_new)
print("F1 score = ",f1)
precision = precision_score(y_test, preds_new)
recall = recall_score(y_test, preds_new)
print("Precision = ", precision)
print("Recall = ", recall)
print("Accuracy = ", accuracy_score(y_test, preds_new))

F1 score =  0.6757281553398058
Precision =  0.5102639296187683
Recall =  1.0
Accuracy =  0.5102639296187683
